<a href="https://colab.research.google.com/github/Burgunthy/Tensorflow2_Practice/blob/main/tf2-10-5-mnist_nn_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# MNist를 사용하여 실습 진행
# softmax를 이용하여 output 구함
# layer의 크기를 바꿔가며 가장 효율적인 모델 구축 -> drop out 사용
# loss: 0.0154 - accuracy: 0.9954
# 거의 비슷한 결과를 냈다. 다른 dataset에서도 사용해 효율을 분석한다

import numpy as np
import random
import tensorflow as tf

In [8]:
learning_rate = 0.001
batch_size = 100
# 한번에 100 개의 dataset 씩 학습
training_epochs = 15
# 15번의 training
nb_classes = 10
# 마지막 softmax 출력 값

drop_rate = 0.3
# drop out 함수 추가

In [9]:
mnist = tf.keras.datasets.mnist
# keras를 통해 mnist 파일 불러오기

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# MNist data load. -> 추후 개인 공부에도 사용

x_train, x_test = x_train / 255.0, x_test / 255.0
# 픽셀을 255로 나눠 0 ~ 1 사이의 값으로 normalization

print(x_train.shape)  # (60000, 28, 28)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])
print(x_train.shape) # (60000, 784)
# 쉬운 학습을 위해 3차원 -> 2차원 변환
# * 을 사용하여 위와 같이 일렬로 만들 수 있다


y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# in tf1, one_hot= True in read_data_sets("MNIST_data/", one_hot=True)
# 직접 one_hot encoding하여 train, test의 결과값을 저장한다

(60000, 28, 28)
(60000, 784)


In [10]:
# Xavier 정규분포 초기값 설정
# stddev = sqrt(2 / (fan_in + fan_out))

tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer='glorot_normal', activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

tf.model.add(tf.keras.layers.Dense(units=nb_classes, kernel_initializer='glorot_normal', activation='softmax'))
# one_hot을 위해 해당 loss 사용

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])
tf.model.summary()

history = tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)
# batch와 epoch을 이와 같이 활용하니 더욱 깔끔하다.
# pytorch보다 괜찮고 깔끔하게 사용 가능

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)              

In [11]:
predictions = tf.model.predict(x_test)
print('Prediction: \n', predictions)

score = tf.model.evaluate(x_train, y_train)
print('Accuracy: ', score[1])
# loss: 0.2455 - accuracy: 0.9320

Prediction: 
 [[4.4343102e-15 3.2559275e-11 1.0330937e-12 ... 9.9999988e-01
  4.3541224e-14 1.7626786e-07]
 [1.0252072e-11 2.8008594e-11 9.9999988e-01 ... 2.4638798e-11
  7.1242690e-10 9.0981899e-13]
 [3.4426372e-20 1.0000000e+00 2.4116841e-15 ... 1.4349799e-11
  1.9597211e-11 1.1621773e-15]
 ...
 [3.0117224e-15 1.0116582e-11 8.0101708e-13 ... 1.4356565e-10
  3.8268512e-12 2.1129176e-07]
 [2.4832565e-26 4.1415108e-26 2.9863789e-25 ... 1.0797138e-27
  9.0497209e-19 1.4017469e-22]
 [5.1816468e-12 2.1065995e-13 1.5296844e-14 ... 4.1310555e-21
  8.6557463e-14 2.1909420e-14]]
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0154 - accuracy: 0.9954
Accuracy:  0.9954000115394592


In [12]:
# test 하는 함수
random.seed(777)

y_predicted = tf.model.predict(x_test)
# x_test를 통해 미리 predict된 list 생성
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    # x_test의 크기 중 random index를 구한다.
    
    print("index: ", random_index,
          "actual y: ", np.argmax(y_test[random_index]),
          "predicted y: ", np.argmax(y_predicted[random_index]))

index:  3757 actual y:  8 predicted y:  8
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  5
index:  7302 actual y:  8 predicted y:  8
